Date : Aug 3, 2020

<h1>Prime location in Lucknow, India for Real Estate investments</h1>
<h7>Dheerendra Kumar Singh</h7><br/>

<h4>Table of Contents</h4>
<ol>
<li>Introduction</li>
<li>Business Problem</li>
<li>Data</li>
<li>Methodology</li>
<li>Analysis</li>
<li>Results and Discussion</li>
<li>Conclusion</li>
</ol>

<h4>Introduction:</h4> 
A Real Estate investor Mr. XYZ from Mumbai, India has decided to expand his business in other parts of country. His business is to Buy, Renovate and Sell Real Estate properties. He decided to expand his business in Lucknow, the capital of UP, India. Since this is a new location for him, he decided to take help from professional data scientist to explore the prime location for Real Estate Investment in Lucknow.<br/>
About Lucknow : <a hreff="https://en.wikipedia.org/wiki/Lucknow">https://en.wikipedia.org/wiki/Lucknow</a>

<h4>Business Problem:</h4>
The task is to locate the hotspot for Real Estate investment in Lucknow, India. 

<h4>Audience:</h4>
Mr. XYZ from Mumbai, India is stakeholder who want to expand his business to new place. Before investing, Mr. XYZ want to explore the neighborhood and would be sure that his investment is profitable. 

<h4>Data acquisition:</h4>
<ol>
<li>Postal code data with coordinates.
Source: IN.zip file from http://www.geonames.org/export/zip/<br/>
    IN.zip file consist of tab seperated text file i.e. IN.txt. The file consists of Country Code, Postal Code, Location Name, State Name, State Code, City Name, District Name and Coordinates</li>
<li>FourSquare API to explore the location</li>
<li>Folium API to visualize result on map.</li>
</ol>

In [92]:
#import library
import pandas as pd
import folium
import foursquare
import requests
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [89]:
#load postal code file
lko_place = pd.read_csv('LKO.csv',encoding='ISO-8859-1')
#pcode.columns = ['CountryCode', 'PIN', 'Place', 'StateName', 'StateCode','City','A', 'District','B','Latitude','Longitude','Accuracy']
lko_place.head()

,Neighborhood,Latitude,Longitude
0,Gomti Nagar,26.844857,80.974458
1,Hazratganj,26.852021,80.939195
2,Mahanagar,26.879082,80.945452
3,Amar Shaheed Path,26.771042,80.951219
4,Indira Nagar,26.888393,80.977364


In [90]:
# create map of New York using latitude and longitude values
lko_lat= 26.848623
lko_lon=80.8724629
map_lko = folium.Map(location=[lko_lat, lko_lon], zoom_start=12)

# add markers to map
for lat, lng, place in zip(lko_place['Latitude'], lko_place['Longitude'], lko_place['Neighborhood']):
    label = '{}, {}'.format(place,'Lucknow')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lko)  
    
map_lko

In [40]:
#CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ' # your Foursquare ID
#CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' # your Foursquare Secret
CLIENT_ID = 'EIAXRTGBTG1V2F1OEMGATO5IV3SVGIXAMGOMHMBMSLIUW5GS' # your Foursquare ID
CLIENT_SECRET = 'PGZ50JQ0PJZO2JI0NHTG2SZJ2DRF43ZEZOIP1GZNVT0LMAHO' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 250

Your credentails:
CLIENT_ID: EIAXRTGBTG1V2F1OEMGATO5IV3SVGIXAMGOMHMBMSLIUW5GS
CLIENT_SECRET:PGZ50JQ0PJZO2JI0NHTG2SZJ2DRF43ZEZOIP1GZNVT0LMAHO


Function that extracts the category of the venue.

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to repeat the same process to all the neighborhoods in Lucknow

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
lko_venues = getNearbyVenues(names=lko_place['Place'],
                                   latitudes=lko_place['Latitude'],
                                   longitudes=lko_place['Longitude']
                                  )
lko_venues.head()

Gomti Nagar
Hazratganj
Mahanagar
Amar Shaheed Path
Indira Nagar
Lucknow Cantt
Jankipuram
Aliganj
Cantonment area
Mall avenue
Governer House
Rajendra Nagar
Charbagh
AP Sen Road


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Gomti Nagar,26.844857,80.974458,Ambedkar Park,26.848229,80.976118,Park
1,Gomti Nagar,26.844857,80.974458,Dezzerts,26.848467,80.973166,Ice Cream Shop
2,Hazratganj,26.852021,80.939195,Dastarkhwan,26.852692,80.936927,Indian Restaurant
3,Hazratganj,26.852021,80.939195,Royal Sky,26.850241,80.940965,Indian Restaurant
4,Hazratganj,26.852021,80.939195,Café Coffee Day,26.849581,80.935043,Coffee Shop


Let's check the size of the resulting dataframe

In [46]:
print(lko_venues.shape)
lko_venues.to_csv('lko_venues.csv')
lko_venues.head()

(39, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Gomti Nagar,26.844857,80.974458,Ambedkar Park,26.848229,80.976118,Park
1,Gomti Nagar,26.844857,80.974458,Dezzerts,26.848467,80.973166,Ice Cream Shop
2,Hazratganj,26.852021,80.939195,Dastarkhwan,26.852692,80.936927,Indian Restaurant
3,Hazratganj,26.852021,80.939195,Royal Sky,26.850241,80.940965,Indian Restaurant
4,Hazratganj,26.852021,80.939195,Café Coffee Day,26.849581,80.935043,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [47]:
lko_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
AP Sen Road,4,4,4,4,4,4
Aliganj,2,2,2,2,2,2
Amar Shaheed Path,1,1,1,1,1,1
Cantonment area,2,2,2,2,2,2
Charbagh,1,1,1,1,1,1
Gomti Nagar,2,2,2,2,2,2
Governer House,6,6,6,6,6,6
Hazratganj,7,7,7,7,7,7
Mahanagar,8,8,8,8,8,8


Let's find out how many unique categories can be curated from all the returned venues

In [48]:
print('There are {} uniques categories.'.format(len(lko_venues['Venue Category'].unique())))

There are 27 uniques categories.


Analyze Each Neighborhood

In [49]:
# one hot encoding
lko_onehot = pd.get_dummies(lko_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lko_onehot['Neighborhood'] = lko_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lko_onehot.columns[-1]] + list(lko_onehot.columns[:-1])
lko_onehot = lko_onehot[fixed_columns]

lko_onehot.head()

,Neighborhood,ATM,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore,Breakfast Spot,Café,...,Indian Restaurant,Indie Movie Theater,Motel,Park,Pizza Place,Pool Hall,Shop & Service,Shopping Mall,Snack Place,Train Station
0,Gomti Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Gomti Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hazratganj,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Hazratganj,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Hazratganj,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [50]:
lko_onehot.shape

(39, 28)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each catego

In [54]:
lko_grouped = lko_onehot.groupby('Neighborhood').mean().reset_index()
lko_grouped

,Neighborhood,ATM,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore,Breakfast Spot,Café,...,Indian Restaurant,Indie Movie Theater,Motel,Park,Pizza Place,Pool Hall,Shop & Service,Shopping Mall,Snack Place,Train Station
0,AP Sen Road,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.0,0.250000,0.000000,0.25
1,Aliganj,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.5,0.000000,...,0.000000,0.000000,0.000000,0.000,0.500000,0.000,0.0,0.000000,0.000000,0.00
2,Amar Shaheed Path,0.000000,1.0,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.00
3,Cantonment area,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.5,0.000000,0.000000,0.00
4,Charbagh,1.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.00
5,Gomti Nagar,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.500,0.000000,0.000,0.0,0.000000,0.000000,0.00
6,Governer House,0.000000,0.0,0.166667,0.000,0.000000,0.000000,0.000,0.0,0.333333,...,0.166667,0.000000,0.000000,0.000,0.166667,0.000,0.0,0.166667,0.000000,0.00
7,Hazratganj,0.000000,0.0,0.000000,0.000,0.142857,0.000000,0.000,0.0,0.000000,...,0.571429,0.000000,0.000000,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.00
8,Mahanagar,0.000000,0.0,0.000000,0.125,0.000000,0.000000,0.125,0.0,0.000000,...,0.000000,0.125000,0.000000,0.125,0.125000,0.125,0.0,0.000000,0.000000,0.00
9,Mall avenue,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,...,0.333333,0.333333,0.333333,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.00


In [ ]:
Let's confirm the new size

In [55]:
lko_grouped.shape

(11, 28)

Let's print each neighborhood along with the top 5 most common venues

In [77]:
num_top_venues = 5

for hood in lko_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = lko_grouped[lko_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AP Sen Road----
               venue  freq
0         Food Court  0.25
1  Fish & Chips Shop  0.25
2      Shopping Mall  0.25
3      Train Station  0.25
4    Bed & Breakfast  0.00


----Aliganj----
            venue  freq
0     Pizza Place   0.5
1  Breakfast Spot   0.5
2             ATM   0.0
3      Food Truck   0.0
4     Snack Place   0.0


----Amar Shaheed Path----
              venue  freq
0  Asian Restaurant   1.0
1               ATM   0.0
2        Food Truck   0.0
3       Snack Place   0.0
4     Shopping Mall   0.0


----Cantonment area----
               venue  freq
0     Shop & Service   0.5
1  Convenience Store   0.5
2                ATM   0.0
3         Food Truck   0.0
4        Snack Place   0.0


----Charbagh----
            venue  freq
0             ATM   1.0
1      Food Truck   0.0
2     Snack Place   0.0
3   Shopping Mall   0.0
4  Shop & Service   0.0


----Gomti Nagar----
            venue  freq
0            Park   0.5
1  Ice Cream Shop   0.5
2             ATM   0.0
3  

Let's put that into a pandas dataframe First, let's write a function to sort the venues in descending order.

In [78]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [109]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lko_grouped['Neighborhood']

for ind in np.arange(lko_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lko_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AP Sen Road,Train Station,Fish & Chips Shop,Food Court,Shopping Mall,Shop & Service,Convenience Store,Asian Restaurant,Bakery,Bar,Bed & Breakfast
1,Aliganj,Pizza Place,Breakfast Spot,Train Station,Fish & Chips Shop,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore
2,Amar Shaheed Path,Asian Restaurant,Train Station,Fish & Chips Shop,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore,Breakfast Spot,Café
3,Cantonment area,Shop & Service,Convenience Store,Train Station,Fish & Chips Shop,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore
4,Charbagh,ATM,Fish & Chips Shop,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore,Breakfast Spot,Café


In [110]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood']=='Mahanagar']

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Mahanagar,Pool Hall,Pizza Place,Park,Bar,Indie Movie Theater,IT Services,Bookstore,Fast Food Restaurant,Train Station,Coffee Shop


In [112]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood']=='Hazratganj']

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Hazratganj,Indian Restaurant,Bed & Breakfast,Coffee Shop,Food Truck,Train Station,Fast Food Restaurant,Asian Restaurant,Bakery,Bar,Boat or Ferry


In [111]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood']=='Governer House']

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Governer House,Café,Shopping Mall,Bakery,Pizza Place,Indian Restaurant,Train Station,Fast Food Restaurant,Asian Restaurant,Bar,Bed & Breakfast


<h4>Methodology:</h4>
Real Estate that are closer to the School, Hospital, Restaurant, Clubs, Gym, Airport, Bus transport, etc are sell at a higher price and having more demand than others. The majority of people want to live close to where they work, shop and go out to enjoy themselves. We will use FourSquare API to explore the venues for each neighborhood


<h4>Analysis</h4>

In [117]:
lko_venues.groupby('Neighborhood').count().sort_values('Neighborhood Latitude', ascending=False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Mahanagar,8,8,8,8,8,8
Hazratganj,7,7,7,7,7,7
Governer House,6,6,6,6,6,6
AP Sen Road,4,4,4,4,4,4
Mall avenue,3,3,3,3,3,3
Rajendra Nagar,3,3,3,3,3,3
Aliganj,2,2,2,2,2,2
Cantonment area,2,2,2,2,2,2
Gomti Nagar,2,2,2,2,2,2


In [113]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AP Sen Road,Train Station,Fish & Chips Shop,Food Court,Shopping Mall,Shop & Service,Convenience Store,Asian Restaurant,Bakery,Bar,Bed & Breakfast
1,Aliganj,Pizza Place,Breakfast Spot,Train Station,Fish & Chips Shop,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore
2,Amar Shaheed Path,Asian Restaurant,Train Station,Fish & Chips Shop,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore,Breakfast Spot,Café
3,Cantonment area,Shop & Service,Convenience Store,Train Station,Fish & Chips Shop,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore
4,Charbagh,ATM,Fish & Chips Shop,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore,Breakfast Spot,Café
5,Gomti Nagar,Park,Ice Cream Shop,Train Station,Fast Food Restaurant,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry,Bookstore
6,Governer House,Café,Shopping Mall,Bakery,Pizza Place,Indian Restaurant,Train Station,Fast Food Restaurant,Asian Restaurant,Bar,Bed & Breakfast
7,Hazratganj,Indian Restaurant,Bed & Breakfast,Coffee Shop,Food Truck,Train Station,Fast Food Restaurant,Asian Restaurant,Bakery,Bar,Boat or Ferry
8,Mahanagar,Pool Hall,Pizza Place,Park,Bar,Indie Movie Theater,IT Services,Bookstore,Fast Food Restaurant,Train Station,Coffee Shop
9,Mall avenue,Motel,Indie Movie Theater,Indian Restaurant,Train Station,Fast Food Restaurant,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Boat or Ferry


<h4>Results and Discussion</h4>

<h4>Conclusion</h4>